# Hello, World
This is the dog breed classifying machine learning model.

In [ ]:
!pip install kaggle
import zipfile
import pandas as pd
import numpy as np
import scipy
# import kagglehub
# import os
# from dotenv import load_dotenv


# load_dotenv()
# print(os.environ.get('KAGGLE_USERNAME'))

# path = kagglehub.dataset_download("jessicali9530/stanford-dogs-dataset")

In [22]:
# Get an image from the user
# Just have user upload to the notebook then modify the file path here eventually
# kaggle.api.datasets_download("jessicali9530", "stanford-dogs-dataset")